# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

S. Torniamenti  ->  S. Torniamenti  |  ['S. Torniamenti']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']


Arxiv has 68 new papers today
          2 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/2 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2512.11059


extracting tarball to tmp_2512.11059... done.


S. Torniamenti  ->  S. Torniamenti  |  ['S. Torniamenti']


Issues with the citations
repeated bibliography entry: Marchant+2019
Retrieving document from  https://arxiv.org/e-print/2512.11292


extracting tarball to tmp_2512.11292...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.11059-b31b1b.svg)](https://arxiv.org/abs/2512.11059) | **Milky Way Globular Clusters: Nurseries for Dynamically-Formed Binary Black Holes**  |
|| F. Angeloni, et al. -- incl., <mark>S. Torniamenti</mark> |
|*Appeared on*| *2025-12-15*|
|*Comments*| *11+3 pages, 8 figures; submitted to A&A*|
|**Abstract**|            We present a novel self-consistent theoretical framework to characterize the formation, evolution, and merger sites of dynamically-formed black hole binaries, with a focus on explaining the most massive events observed by the LIGO-Virgo-KAGRA Collaboration. Our approach couples the galaxy formation model GAMESH with cluster population synthesis codes to trace the cosmic evolution of globular clusters simultaneously with mergers of massive black holes. Our reference model, which includes prescriptions for both cluster formation and disruption depending on properties of specific galaxies, accurately reproduces the observed age-mass distribution of the Milky Way globular clusters. We find that approximately 30% of the globular clusters observed in our galaxy's halo may have originated from satellite galaxies of the Milky Way. We confirm that hierarchical black hole mergers provide a significant contribution to the formation of black holes in and above the pair-instability mass gap. However, quantifying their contribution is challenging, as different population synthesis codes yield divergent results in terms of black hole mass function and merger rates. Furthermore, we characterize the host galaxies where massive black holes form in terms of their dark matter, stellar mass, and metallicity. Ultimately, we demonstrate that the merger and birth rate densities of binary black holes increase with redshift till z = 5. This cosmic evolution is a crucial signature with significant implications for future detectors like the LISA, the Einstein Telescope and Cosmic Explorer, which will be capable to probe the high-redshift Universe.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.11292-b31b1b.svg)](https://arxiv.org/abs/2512.11292) | **CPI-C: Cool Planet Imaging Coronagraph on Chinese Space Station Survey Telescope**  |
|| J. Dou, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-12-15*|
|*Comments*| *41 pages, 27 figures, accepted by Research in Astronomy and Astrophysics*|
|**Abstract**|            Cool Planet Imaging Coronagraph (CPI-C) on Chinese Space Station Survey Telescope (CSST) is proposed to direct image the cool planets around nearby solar-type stars (within 40 pc). The core scientific objective of CPI-C is to conduct high-contrast directly imaging surveys of exoplanets ranging in size from Neptune-like to Jupiter-like, located at separations of 0.5 to 5 AU from their host stars, and to perform systematic spectroscopic analysis of the detected planets through high-precision multi-band photometry. CPI-C employs a step-transmission apodization technique to suppress the diffraction noises from the telescope pupil and a precise phase correction technique to eliminate the speckle noises due to imperfections of the optical surfaces. The contrast requirement is better than $10^{-8}$ at an inner working angle (IWA) of $3-4\lambda/D$, in the visible wavelength from 600 nm to 900 nm. CPI-C will be the first space-based instrument capable of directly imaging the reflection light from the cool exoplanets in the visible wavelength enabling the measurement of key physical parameters such as the effective temperature, surface gravity, radius, mass, and other key parameters. The potential observation results will significantly contribute to further understand the formation and evolution mechanisms of planets, which will also lay a solid foundation for future confirmation of the Earth-twins in the next generation space flagship missions.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2512.11059/./figures/M1vsM2.png', 'tmp_2512.11059/./figures/BHB_MergerRateDensity.png', 'tmp_2512.11059/./figures/BHB_BirthRateDensity.png', 'tmp_2512.11059/./figures/SurvivingClusters_MetallicityvsMass.png']
copying  tmp_2512.11059/./figures/M1vsM2.png to _build/html/
copying  tmp_2512.11059/./figures/BHB_MergerRateDensity.png to _build/html/
copying  tmp_2512.11059/./figures/BHB_BirthRateDensity.png to _build/html/
copying  tmp_2512.11059/./figures/SurvivingClusters_MetallicityvsMass.png to _build/html/
exported in  _build/html/2512.11059.md
    + _build/html/tmp_2512.11059/./figures/M1vsM2.png
    + _build/html/tmp_2512.11059/./figures/BHB_MergerRateDensity.png
    + _build/html/tmp_2512.11059/./figures/BHB_BirthRateDensity.png
    + _build/html/tmp_2512.11059/./figures/SurvivingClusters_MetallicityvsMass.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\msun}{{\rm M}_\odot}$
$\newcommand{\fastcluster}{\textsc{fastcluster} }$
$\newcommand$
$\newcommand$
$\newcommand{\RV}[1]{{\color{cyan}{#1}}}$
$\newcommand{\KK}[1]{{\color{red}{[KK: #1]}}}$
$\newcommand{\eb}[1]{{\color{blue}{[EB: #1]}}}$</div>



<div id="title">

# Milky Way Globular Clusters: Nurseries for Dynamically-Formed Binary Black Holes

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2512.11059-b31b1b.svg)](https://arxiv.org/abs/2512.11059)<mark>Appeared on: 2025-12-15</mark> -  _11+3 pages, 8 figures; submitted to A&A_

</div>
<div id="authors">

F. Angeloni, et al. -- incl., <mark>S. Torniamenti</mark>

</div>
<div id="abstract">

**Abstract:** We present a novel self-consistent theoretical framework to characterize the formation, evolution, and merger sites of dynamically-formed black hole binaries, with a focus on explaining the most massive events observed by the LIGO-Virgo-KAGRA Collaboration. Our approach couples the galaxy formation model GAMESH with cluster population synthesis codes to trace the cosmic evolution of globular clusters simultaneously with mergers of massive black holes. \ Our reference model, which includes prescriptions for both cluster formation and disruption depending on properties of specific galaxies, accurately reproduces the observed age-mass distribution of the Milky Way's globular clusters. We find that approximately 30 \% of the globular clusters observed in our galaxy's halo may have originated from satellite galaxies of the Milky Way. We confirm that hierarchical black hole mergers provide a significant contribution to the formation of black holes in and above the pair-instability mass gap. However, quantifying their contribution is challenging, as different population synthesis codes yield divergent results in terms of black hole mass function and merger rates. \ Furthermore, we characterize the host galaxies where massive black holes form in terms of their dark matter, stellar mass, and metallicity. \ Ultimately, we demonstrate that the merger and birth rate densities of binary black holes increase with redshift till _z_ = 5. This cosmic evolution is a crucial signature with significant implications for future detectors like the LISA, the Einstein Telescope and Cosmic Explorer, which will be capable to probe the high-redshift Universe.

</div>

<div id="div_fig1">

<img src="tmp_2512.11059/./figures/M1vsM2.png" alt="Fig2" width="100%"/>

**Figure 2. -** Primary ($M_1$) and secondary ($M_2$) BH mass distribution of observable dynamically-formed BBHs within our LG-like volume.
    Red dots show the estimated median  values of the most massive GW events (M$_{1}> 50$ M$_\odot$) detected by the LVK collaboration up to O4a, with error bars representing the 90\% credible intervals. Dashed lines in histograms display the mass distributions of all BBH mergers that occur in our simulation.
     (*fig:M1M2Plot*)

</div>
<div id="div_fig2">

<img src="tmp_2512.11059/./figures/BHB_MergerRateDensity.png" alt="Fig4.1" width="50%"/><img src="tmp_2512.11059/./figures/BHB_BirthRateDensity.png" alt="Fig4.2" width="50%"/>

**Figure 4. -** (a) Redshift evolution of dynamically-formed BBH merger rate density, as a function of primary BH mass ($M_1$). Red curves show BBH with $M_1\leq100$ M$_\odot$, green lines indicate MBBHs with $100$ M$_\odot <$ $M_1\leq1000$ M$_\odot$, and violet histograms illustrate IMBHs with $M_1>1000$ M$_\odot$. Blue star with error bars represents the value provided by the LVK collaboration \citep{GWTC4v2+2025}; (b) Redshift evolution of dynamically-formed BBH birth rate density, as a function of primary BH mass ($M_1$).  Red curves show BBH with $M_1\leq100$ M$_\odot$, green lines indicate MBBHs with $100$ M$_\odot <$ $M_1\leq1000$ M$_\odot$, and violet histograms illustrate IMBHs with $M_1>1000$ M$_\odot$. (*fig:BHB_MergerRateDensity*)

</div>
<div id="div_fig3">

<img src="tmp_2512.11059/./figures/SurvivingClusters_MetallicityvsMass.png" alt="Fig5" width="100%"/>

**Figure 5. -** Mass-metallicity distribution of surviving MW GCs. The color bar indicates the central stellar number density of GCs. The red dashed line in the top histogram highlights the mass distribution of the observed GCs in our MW \citep{Kruijssen+2015}.
     (*fig:GCMassDistribution*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2512.11059"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

139  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

9  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
